# Pandas Basics

In [1]:
import pandas as pd


In [3]:
df = pd.DataFrame([['Abhigyan',23,'Allahabad']],columns=['Name','Age','City'],index=[1])
df.head()

,Name,Age,City
1,Abhigyan,23,Allahabad


In [4]:
df.loc[2] = ['Rahul',24,'Kanpur']

In [5]:
df.head()

,Name,Age,City
1,Abhigyan,23,Allahabad
2,Rahul,24,Kanpur


In [14]:
#task1
def neighbourhood_with_highest_median_price_diff(df_listings: pd.DataFrame, df_reviews: pd.DataFrame) -> str:
    # Group by neighborhood and superhost status, calculate median prices
    grouped = df_listings.groupby(['neighbourhood_cleansed', 'host_is_superhost'])['price'].median().unstack()
    
    # Calculate price difference (superhost - non-superhost)
    # True column is superhost prices, False column is non-superhost prices
    price_diff = grouped[True] - grouped[False]
    
    # Find neighborhood with maximum difference
    return price_diff.idxmax()

In [15]:
# task2
def find_strongest_correlation_with_price(df_listings: pd.DataFrame, df_reviews: pd.DataFrame):
    review_score_cols = [col for col in df_listings.columns if 'review_scores' in col]
    if 'price' in df_listings.columns:
        correlations = df_listings[review_score_cols].corrwith(df_listings['price'])
        strongest_corr = correlations.abs().idxmax()
        # or 
        strongest_corr = correlations.abs().sort_values(ascending=False).index[0]
        return strongest_corr
    else:
        raise ValueError("The 'price' column is not present in the df_listings DataFrame.")


In [16]:
#task3
def calculate_professional_host_price_difference(df_listings: pd.DataFrame, df_reviews: pd.DataFrame) -> float:
    # Count unique neighborhoods per host
    locations_per_host = df_listings.groupby('host_id')['neighbourhood_cleansed'].nunique()
    
    # Identify professional hosts (more than 5 locations)
    professional_hosts = locations_per_host[locations_per_host > 5].index
    
    # Create a boolean mask for professional hosts
    df_listings['is_professional'] = df_listings['host_id'].isin(professional_hosts)
    
    # Calculate average price for professional and non-professional hosts
    avg_prices = df_listings.groupby('is_professional')['price'].mean()
    
    # Calculate the difference (professional - non-professional)
    price_difference = avg_prices[True] - avg_prices[False]
    
    return price_difference

In [17]:
#task4
def calculate_entire_home_price_premium(df_listings: pd.DataFrame, df_reviews: pd.DataFrame) -> float:
    # Identify entire homes/apartments, accounting for possible separate categories
    entire_property_types = ['Entire home', 'Entire apartment', 'Entire home/apt']
    entire_home_mask = df_listings['room_type'].str.contains('|'.join(entire_property_types), case=False)
    
    # Group by neighborhood and calculate median prices for entire properties vs others
    neighborhood_prices = df_listings.groupby(['neighbourhood_cleansed', entire_home_mask])['price'].median().unstack()
    
    # Calculate premium (difference between entire properties and other types)
    # True column is entire properties, False column is other types
    price_premiums = neighborhood_prices[True] - neighborhood_prices[False]
    
    # Calculate average premium across all neighborhoods
    average_premium = price_premiums.mean()
    
    return average_premium

In [18]:
#task5
def find_best_listing_for_revenue(df_listings: pd.DataFrame, df_reviews: pd.DataFrame) -> int:
    # Filter listings with minimum_nights <= 7
    eligible_listings = df_listings[df_listings['minimum_nights'] <= 7]
    
    # Count reviews per listing in the last 12 months
    # Assuming reviews DataFrame has a datetime column for the review date
    df_reviews['review_date'] = pd.to_datetime(df_reviews['date'])
    last_year = df_reviews['review_date'].max() - pd.DateOffset(months=12)
    recent_reviews = df_reviews[df_reviews['review_date'] >= last_year]
    
    # Count reviews per listing
    reviews_count = recent_reviews.groupby('listing_id').size().reset_index(name='review_count')
    
    # Merge review counts with eligible listings
    revenue_analysis = eligible_listings.merge(reviews_count, 
                                             left_on='id', 
                                             right_on='listing_id', 
                                             how='left')
    
    # Fill missing review counts with 0
    revenue_analysis['review_count'] = revenue_analysis['review_count'].fillna(0)
    
    # Calculate estimated bookings (assuming 60% review rate)
    revenue_analysis['estimated_bookings'] = revenue_analysis['review_count'] / 0.6
    
    # Calculate expected revenue
    revenue_analysis['expected_revenue'] = revenue_analysis['estimated_bookings'] * revenue_analysis['price']
    
    # Find listing with highest expected revenue
    best_listing = revenue_analysis.loc[revenue_analysis['expected_revenue'].idxmax()]
    
    return int(best_listing['id'])

In [19]:
#task6
def get_avg_review_score_difference(df_listings: pd.DataFrame, df_reviews: pd.DataFrame) -> float:
    # Group by superhost status and calculate mean review scores
    avg_scores = df_listings.groupby('host_is_superhost')['review_scores_rating'].mean()
    
    # Calculate difference (superhost scores - normal host scores)
    score_difference = avg_scores[True] - avg_scores[False]
    
    return score_difference

In [22]:
#task7
def get_second_strongest_host_correlation(df_listings: pd.DataFrame, df_reviews: pd.DataFrame) -> str:
    # List of host attributes to check
    host_attributes = [
        'host_since',
        'host_listings_count',
        'host_identity_verified',
        'calculated_host_listings_count',
        'host_is_superhost'
    ]
    
    # Calculate correlations between host attributes and number of reviews
    correlations = df_listings[host_attributes].corrwith(df_listings['number_of_reviews'])
    
    # Sort correlations by absolute value to handle both positive and negative correlations
    sorted_correlations = correlations.abs().sort_values(ascending=False)
    
    # Get the second strongest correlation
    second_strongest = sorted_correlations.index[1]
    
    return second_strongest

In [1]:
# Dummy values for df_listings
df_listings = pd.DataFrame({
    'room_type': ['Entire home', 'Private room', 'Entire apartment', 'Shared room', 'Entire home/apt']
})

# Define entire_property_types and create entire_home_mask
entire_property_types = ['Entire home', 'Entire apartment', 'Entire home/apt']
entire_home_mask = df_listings['room_type'].str.contains('|'.join(entire_property_types), case=False)

print(entire_home_mask)

NameError: name 'pd' is not defined